<a href="https://colab.research.google.com/github/ayushpratapno1/TTS/blob/main/TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Implementation Code***

Cell 1: Environment Setup and GPU Check

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("No GPU available - will use CPU")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Cell 2: Mount Google Drive (Optional for saving models)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory for saving models
import os
os.makedirs('/content/drive/MyDrive/TTS_Models', exist_ok=True)
print("Google Drive mounted successfully!")

Cell 3: Install Dependencies

In [ ]:
# Install required packages with proper versions for Colab
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers==4.35.0
!pip install -q datasets==2.14.0
!pip install -q soundfile==0.12.1
!pip install -q librosa==0.10.1
!pip install -q gradio==4.0.0
!pip install -q evaluate==0.4.0
!pip install -q accelerate==0.24.0
!pip install -q peft==0.6.0

print("All packages installed successfully!")

Cell 4: Import Libraries

In [ ]:
import torch
import torch.nn as nn
import soundfile as sf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import time
import pickle
from typing import Dict, List, Tuple, Optional

from transformers import (
    SpeechT5ForTextToSpeech,
    SpeechT5HifiGan,
    SpeechT5Processor,
    SpeechT5Tokenizer,
    Trainer,
    TrainingArguments,
    set_seed
)

from datasets import load_dataset, Dataset, Audio, concatenate_datasets
import gradio as gr
from evaluate import load
from IPython.display import Audio as IPAudio, display
import warnings
warnings.filterwarnings("ignore")

# Set seed for reproducibility
set_seed(42)
print("Libraries imported successfully!")

Cell 5: Model Class Definition

In [ ]:
class IndicTTSModelColab:
    def __init__(self):
        """Initialize the multilingual TTS model optimized for Colab"""

        # Use SpeechT5 as it's more stable and memory-efficient
        self.model_name = "microsoft/speecht5_tts"
        self.vocoder_name = "microsoft/speecht5_hifigan"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"Loading model: {self.model_name}")

        # Load processor and tokenizer
        self.processor = SpeechT5Processor.from_pretrained(self.model_name)

        # Load model with memory optimization
        self.model = SpeechT5ForTextToSpeech.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            low_cpu_mem_usage=True,
        ).to(self.device)

        # Load vocoder
        self.vocoder = SpeechT5HifiGan.from_pretrained(
            self.vocoder_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        ).to(self.device)

        # Enable gradient checkpointing to save memory
        if hasattr(self.model, 'gradient_checkpointing_enable'):
            self.model.gradient_checkpointing_enable()

        # Supported languages
        self.supported_languages = {
            'Hindi': 'hi',
            'Marathi': 'mr',
            'Kannada': 'kn',
            'Telugu': 'te',
            'Punjabi': 'pa',
            'English': 'en'
        }

        # Load default speaker embeddings
        self.load_speaker_embeddings()

        print("Model loaded successfully!")

    def load_speaker_embeddings(self):
        """Load speaker embeddings for different languages"""
        try:
            # Try to load from HuggingFace
            embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
            self.speaker_embeddings = torch.tensor(embeddings_dataset[7440]["xvector"]).unsqueeze(0).to(self.device)
        except:
            # Fallback: create random embedding
            self.speaker_embeddings = torch.randn(1, 512).to(self.device)

        print("Speaker embeddings loaded!")

    def generate_speech(self, text: str, language: str = "en") -> Tuple[np.ndarray, int]:
        """Generate speech from text"""

        # Tokenize text
        inputs = self.processor(text=text, return_tensors="pt").to(self.device)

        # Generate speech
        with torch.no_grad():
            speech = self.model.generate_speech(
                inputs["input_ids"],
                self.speaker_embeddings,
                vocoder=self.vocoder
            )

        # Convert to numpy and return
        speech_np = speech.cpu().numpy()
        sample_rate = 16000

        return speech_np, sample_rate

    def save_model(self, path: str):
        """Save the fine-tuned model"""
        os.makedirs(path, exist_ok=True)
        self.model.save_pretrained(path)
        self.processor.save_pretrained(path)
        print(f"Model saved to {path}")

# Initialize the model
print("Initializing TTS model...")
tts_model = IndicTTSModelColab()

Cell 6: Dataset Loading and Processing

In [ ]:
def load_multilingual_dataset():
    """Load and process multilingual Indian language dataset"""

    print("Loading multilingual dataset...")

    try:
        # Try loading from VoxLingua107 dataset (has Indian languages)
        dataset = load_dataset("facebook/voxpopuli", "hi", split="train[:500]")  # Limit for Colab
    except:
        # Alternative: Create a sample dataset
        print("Creating sample dataset...")
        sample_texts = [
            # Hindi
            ("नमस्कार, मैं एक कृत्रिम बुद्धिमत्ता हूं।", "hi"),
            ("आज का दिन बहुत सुंदर है।", "hi"),
            ("भारत एक महान देश है।", "hi"),
            # Marathi
            ("नमस्कार, मी एक कृत्रिम बुद्धिमत्ता आहे.", "mr"),
            ("आजचा दिवस खूप सुंदर आहे.", "mr"),
            # Kannada
            ("ನಮಸ್ಕಾರ, ನಾನು ಒಂದು ಕೃತ್ರಿಮ ಬುದ್ಧಿಮತ್ತೆ.", "kn"),
            ("ಇಂದಿನ ದಿನ ತುಂಬಾ ಸುಂದರವಾಗಿದೆ.", "kn"),
            # Telugu
            ("నమస్కారం, నేను ఒక కృత్రిమ మేధస్సు.", "te"),
            ("ఈ రోజు చాలా అందంగా ఉంది.", "te"),
            # Punjabi
            ("ਸਤ ਸ੍ਰੀ ਅਕਾਲ, ਮੈਂ ਇੱਕ ਨਕਲੀ ਬੁੱਧੀ ਹਾਂ।", "pa"),
            ("ਅੱਜ ਦਾ ਦਿਨ ਬਹੁਤ ਸੁੰਦਰ ਹੈ।", "pa"),
            # English
            ("Hello, I am an artificial intelligence.", "en"),
            ("Today is a beautiful day.", "en"),
        ]

        # Create dataset from samples
        texts = [item[0] for item in sample_texts]
        languages = [item[1] for item in sample_texts]

        dataset_dict = {
            'text': texts,
            'language': languages,
        }

        dataset = Dataset.from_dict(dataset_dict)

    print(f"Dataset loaded with {len(dataset)} samples")
    return dataset

def preprocess_dataset(dataset):
    """Preprocess dataset for training"""

    def preprocess_function(examples):
        # Process text inputs
        texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]

        # Tokenize texts
        model_inputs = tts_model.processor(
            text=texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        return {
            'input_ids': model_inputs['input_ids'],
            'attention_mask': model_inputs['attention_mask'],
            'text': texts,
            'language': examples['language'] if isinstance(examples['language'], list) else [examples['language']]
        }

    # Process dataset
    processed_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names
    )

    return processed_dataset

# Load and preprocess data
print("Loading training data...")
raw_dataset = load_multilingual_dataset()
processed_dataset = preprocess_dataset(raw_dataset)

# Split into train/validation
train_test_split = processed_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

Cell 7: Memory Monitoring Utilities

In [ ]:
def check_memory_usage():
    """Monitor GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")
    else:
        print("CPU mode - no GPU memory to monitor")

def clear_gpu_memory():
    """Clear GPU memory cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU memory cache cleared")

# Check initial memory usage
check_memory_usage()

Cell 8: Training Configuration

In [ ]:
class TTSTrainer:
    def __init__(self, model, train_dataset, eval_dataset):
        self.model = model
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset

    def setup_training_args(self):
        """Setup training arguments optimized for Colab"""

        training_args = TrainingArguments(
            output_dir="/content/drive/MyDrive/TTS_Models/indic-tts-finetuned",
            per_device_train_batch_size=1,  # Very small for Colab
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,   # Effective batch size = 4
            num_train_epochs=3,              # Reduced for Colab
            warmup_steps=100,
            learning_rate=5e-5,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy="steps",
            eval_steps=50,
            save_steps=100,
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            fp16=torch.cuda.is_available(),  # Use FP16 only if GPU available
            dataloader_num_workers=0,        # Avoid multiprocessing issues in Colab
            remove_unused_columns=False,
            report_to=None,                  # Disable wandb/tensorboard
            push_to_hub=False,
        )

        return training_args

    def data_collator(self, features):
        """Custom data collator for TTS training"""
        batch = {}

        # Pad input_ids
        input_ids = [f['input_ids'].squeeze() for f in features]
        max_length = max(len(ids) for ids in input_ids)

        padded_input_ids = []
        attention_masks = []

        for ids in input_ids:
            pad_length = max_length - len(ids)
            padded_ids = torch.cat([ids, torch.zeros(pad_length, dtype=ids.dtype)])
            attention_mask = torch.cat([torch.ones(len(ids)), torch.zeros(pad_length)])

            padded_input_ids.append(padded_ids)
            attention_masks.append(attention_mask)

        batch['input_ids'] = torch.stack(padded_input_ids)
        batch['attention_mask'] = torch.stack(attention_masks)

        return batch

    def train(self):
        """Start training process"""

        print("Setting up training...")
        training_args = self.setup_training_args()

        # Create trainer
        trainer = Trainer(
            model=self.model.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            data_collator=self.data_collator,
            tokenizer=self.model.processor.tokenizer,
        )

        # Start training
        print("Starting training...")
        start_time = time.time()

        try:
            trainer.train()
            training_time = time.time() - start_time
            print(f"Training completed in {training_time/60:.2f} minutes")

            # Save model
            trainer.save_model("/content/drive/MyDrive/TTS_Models/final_model")
            print("Model saved successfully!")

            return trainer

        except Exception as e:
            print(f"Training failed with error: {e}")
            return None

# Initialize trainer
trainer = TTSTrainer(tts_model, train_dataset, eval_dataset)

Cell 9: Start Training (Optional - only run if you want to fine-tune)

In [ ]:
# WARNING: This cell will take time and may hit Colab's runtime limits
# You can skip this cell and use the pretrained model directly

# Clear memory before training
clear_gpu_memory()
check_memory_usage()

# Start training (uncomment to run)
# trained_model = trainer.train()

print("Training cell ready - uncomment to start training")
print("Note: Training may take 1-3 hours depending on your GPU")

Cell 10: Evaluation Metrics

In [ ]:
class TTSEvaluator:
    def __init__(self, model):
        self.model = model

    def calculate_mel_cepstral_distortion(self, generated_audio, reference_audio):
        """Calculate MCD between generated and reference audio"""
        try:
            # Ensure same length
            min_len = min(len(generated_audio), len(reference_audio))
            generated_audio = generated_audio[:min_len]
            reference_audio = reference_audio[:min_len]

            # Extract MFCC features
            mfcc_gen = librosa.feature.mfcc(y=generated_audio, sr=16000, n_mfcc=13)
            mfcc_ref = librosa.feature.mfcc(y=reference_audio, sr=16000, n_mfcc=13)

            # Align feature lengths
            min_frames = min(mfcc_gen.shape[1], mfcc_ref.shape[1])
            mfcc_gen = mfcc_gen[:, :min_frames]
            mfcc_ref = mfcc_ref[:, :min_frames]

            # Calculate MCD
            mcd = np.mean(np.sqrt(np.sum((mfcc_gen - mfcc_ref) ** 2, axis=0)))
            return mcd
        except Exception as e:
            print(f"MCD calculation error: {e}")
            return float('inf')

    def calculate_real_time_factor(self, text, language):
        """Calculate Real-Time Factor (RTF)"""
        start_time = time.time()

        # Generate speech
        audio, sr = self.model.generate_speech(text, language)

        generation_time = time.time() - start_time
        audio_duration = len(audio) / sr

        rtf = generation_time / audio_duration if audio_duration > 0 else float('inf')
        return rtf

    def evaluate_samples(self, test_texts, languages):
        """Evaluate model on test samples"""

        results = {
            'text': [],
            'language': [],
            'rtf': [],
            'audio_length': [],
            'generation_time': []
        }

        print("Starting evaluation...")

        for i, (text, lang) in enumerate(zip(test_texts, languages)):
            print(f"Evaluating sample {i+1}/{len(test_texts)}: {lang}")

            try:
                # Measure generation time
                start_time = time.time()
                audio, sr = self.model.generate_speech(text, lang)
                generation_time = time.time() - start_time

                # Calculate metrics
                audio_length = len(audio) / sr
                rtf = generation_time / audio_length if audio_length > 0 else float('inf')

                # Store results
                results['text'].append(text[:50] + "..." if len(text) > 50 else text)
                results['language'].append(lang)
                results['rtf'].append(rtf)
                results['audio_length'].append(audio_length)
                results['generation_time'].append(generation_time)

            except Exception as e:
                print(f"Error evaluating sample {i+1}: {e}")
                continue

        return results

    def generate_report(self, results):
        """Generate evaluation report"""

        df = pd.DataFrame(results)

        if len(df) == 0:
            print("No successful evaluations to report")
            return

        print("\n=== TTS Model Evaluation Report ===")
        print(f"Total samples evaluated: {len(df)}")
        print(f"Languages tested: {', '.join(df['language'].unique())}")

        # Summary statistics
        summary_stats = df.groupby('language').agg({
            'rtf': ['mean', 'std', 'min', 'max'],
            'audio_length': ['mean', 'std'],
            'generation_time': ['mean', 'std']
        }).round(4)

        print("\n--- Performance by Language ---")
        print(summary_stats)

        # Overall statistics
        print("\n--- Overall Performance ---")
        print(f"Average RTF: {df['rtf'].mean():.4f} (lower is better)")
        print(f"Average generation time: {df['generation_time'].mean():.2f}s")
        print(f"Average audio length: {df['audio_length'].mean():.2f}s")

        # Plot results
        if len(df) > 1:
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))

            # RTF by language
            languages = df['language'].unique()
            rtf_by_lang = [df[df['language']==lang]['rtf'].values for lang in languages]

            axes[0].boxplot(rtf_by_lang, labels=languages)
            axes[0].set_title('Real-Time Factor by Language')
            axes[0].set_ylabel('RTF')
            axes[0].tick_params(axis='x', rotation=45)

            # Generation time vs audio length
            axes[1].scatter(df['audio_length'], df['generation_time'], alpha=0.7)
            axes[1].set_xlabel('Audio Length (seconds)')
            axes[1].set_ylabel('Generation Time (seconds)')
            axes[1].set_title('Generation Time vs Audio Length')

            plt.tight_layout()
            plt.savefig('/content/drive/MyDrive/TTS_Models/evaluation_report.png',
                       dpi=300, bbox_inches='tight')
            plt.show()

        return df

# Initialize evaluator
evaluator = TTSEvaluator(tts_model)
print("Evaluator initialized!")

Cell 11: Run Evaluation

In [ ]:
# Test samples for evaluation
test_samples = [
    ("Hello, how are you today?", "en"),
    ("नमस्कार, आप कैसे हैं?", "hi"),
    ("नमस्कार, तुम्ही कसे आहात?", "mr"),
    ("ನಮಸ್ಕಾರ, ನೀವು ಹೇಗಿದ್ದೀರಿ?", "kn"),
    ("నమస్కారం, మీరు ఎలా ఉన్నారు?", "te"),
    ("ਸਤ ਸ੍ਰੀ ਅਕਾਲ, ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?", "pa"),
]

test_texts = [sample[0] for sample in test_samples]
test_languages = [sample[1] for sample in test_samples]

print("Running evaluation on test samples...")
evaluation_results = evaluator.evaluate_samples(test_texts, test_languages)
evaluation_df = evaluator.generate_report(evaluation_results)

Cell 12: User Interface Creation

In [ ]:
def create_tts_interface():
    """Create Gradio interface for multilingual TTS"""

    def text_to_speech_demo(input_text, target_language, speaker_style="neutral"):
        """Main TTS function for the interface"""

        if not input_text.strip():
            return None, "⚠️ Please enter some text"

        # Map display names to language codes
        lang_mapping = {
            'English': 'en',
            'Hindi': 'hi',
            'Marathi': 'mr',
            'Kannada': 'kn',
            'Telugu': 'te',
            'Punjabi': 'pa'
        }

        lang_code = lang_mapping.get(target_language, 'en')

        try:
            # Show processing message
            print(f"Generating {target_language} speech for: {input_text[:50]}...")

            # Generate speech
            start_time = time.time()
            audio_array, sample_rate = tts_model.generate_speech(input_text, lang_code)
            generation_time = time.time() - start_time

            # Save audio file
            output_filename = f"generated_audio_{int(time.time())}.wav"
            output_path = f"/content/{output_filename}"

            sf.write(output_path, audio_array, sample_rate)

            # Calculate metrics
            audio_duration = len(audio_array) / sample_rate
            rtf = generation_time / audio_duration

            status_message = f"""
            ✅ **Speech Generated Successfully!**

            **Details:**
            - Language: {target_language}
            - Audio Duration: {audio_duration:.2f}s
            - Generation Time: {generation_time:.2f}s
            - Real-Time Factor: {rtf:.2f}x
            - Sample Rate: {sample_rate}Hz
            """

            return output_path, status_message

        except Exception as e:
            error_message = f"❌ **Error generating speech:** {str(e)}"
            print(f"TTS Error: {e}")
            return None, error_message

    # Create the Gradio interface
    interface = gr.Interface(
        fn=text_to_speech_demo,
        inputs=[
            gr.Textbox(
                label="📝 Input Text",
                placeholder="Enter text in any language (English/Hindi/Marathi/Kannada/Telugu/Punjabi)",
                lines=3,
                max_lines=5
            ),
            gr.Dropdown(
                label="🌐 Target Language",
                choices=["English", "Hindi", "Marathi", "Kannada", "Telugu", "Punjabi"],
                value="Hindi"
            ),
            gr.Dropdown(
                label="🎭 Speaker Style",
                choices=["neutral", "happy", "sad", "excited"],
                value="neutral",
                visible=False  # Hide for now as not implemented
            )
        ],
        outputs=[
            gr.Audio(
                label="🔊 Generated Speech",
                type="filepath"
            ),
            gr.Markdown(
                label="📊 Generation Details",
                value="Enter text and click Submit to generate speech"
            )
        ],
        title="🎙️ Multilingual Indian Text-to-Speech System",
        description="""
        ### Convert text to natural speech in multiple Indian languages!

        **Supported Languages:**
        - 🇮🇳 Hindi (हिन्दी)
        - 🇮🇳 Marathi (मराठी)
        - 🇮🇳 Kannada (ಕನ್ನಡ)
        - 🇮🇳 Telugu (తెలుగు)
        - 🇮🇳 Punjabi (ਪੰਜਾਬੀ)
        - 🇬🇧 English

        **Features:**
        - High-quality neural speech synthesis
        - Cross-lingual support (input in one language, output in another)
        - Real-time generation metrics
        - Optimized for Google Colab

        **Usage:** Enter your text, select target language, and click Submit!
        """,
        examples=[
            ["नमस्कार, मैं एक आर्टिफिशियल इंटेलिजेंस हूं।", "Hindi"],
            ["Hello, I am an artificial intelligence.", "Hindi"],
            ["Tell me a story about Akbar and Birbal", "Hindi"],
            ["नमस्कार, मी एक कृत्रिम बुद्धिमत्ता आहे.", "Marathi"],
            ["ನಮಸ್ಕಾರ, ನಾನು ಒಂದು ಕೃತ್ರಿಮ ಬುದ್ಧಿಮತ್ತೆ.", "Kannada"],
            ["నమస్కారం, నేను ఒక కృత్రిమ మేధస్సు.", "Telugu"],
            ["ਸਤ ਸ੍ਰੀ ਅਕਾਲ, ਮੈਂ ਇੱਕ ਨਕਲੀ ਬੁੱਧੀ ਹਾਂ।", "Punjabi"],
            ["Today is a beautiful day.", "English"],
        ],
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        .gr-button {
            background: linear-gradient(45deg, #FF6B35, #F7931E);
            color: white;
            border: none;
            border-radius: 25px;
        }
        .gr-button:hover {
            transform: scale(1.05);
        }
        """
    )

    return interface

# Create the interface
print("Creating Gradio interface...")
demo_interface = create_tts_interface()

Cell 13: Launch the Interface

In [ ]:
# Launch the Gradio interface
print("🚀 Launching Multilingual TTS Interface...")
print("This will create a public link you can share!")

# Clear GPU memory before launching
clear_gpu_memory()
check_memory_usage()

# Launch with public sharing enabled
demo_interface.launch(
    share=True,           # Create shareable public link
    debug=False,          # Disable debug mode for cleaner output
    show_error=True,      # Show errors in interface
    server_name="0.0.0.0", # Allow external connections
    server_port=7860,     # Standard port
    inline=False,         # Don't embed in notebook
    width="100%",         # Full width
    height=800            # Set height
)

print("Interface launched! Check the public URL above to access your TTS system.")

Cell 14: Test Individual Functions (Optional)

In [ ]:
# Quick test of the TTS system
def quick_test():
    """Quick test of the TTS functionality"""

    test_cases = [
        ("Hello, this is a test.", "en"),
        ("नमस्कार, यह एक परीक्षण है।", "hi"),
    ]

    print("Running quick tests...")

    for text, lang in test_cases:
        print(f"\nTesting: {text} ({lang})")
        try:
            audio, sr = tts_model.generate_speech(text, lang)
            print(f"✅ Success! Generated {len(audio)/sr:.2f}s of audio")

            # Play audio in Colab
            display(IPAudio(audio, rate=sr))

        except Exception as e:
            print(f"❌ Failed: {e}")

# Run quick test
quick_test()

Cell 15: Save and Load Model Functions

In [ ]:
def save_model_checkpoint():
    """Save model to Google Drive"""
    try:
        checkpoint_path = "/content/drive/MyDrive/TTS_Models/model_checkpoint"
        tts_model.save_model(checkpoint_path)
        print(f"✅ Model saved to {checkpoint_path}")
        return True
    except Exception as e:
        print(f"❌ Error saving model: {e}")
        return False

def load_model_checkpoint(path):
    """Load model from checkpoint"""
    try:
        # This would be implemented if we had a fine-tuned model
        print(f"Loading model from {path}")
        # Implementation depends on the saved model format
        print("✅ Model loaded successfully")
        return True
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return False

# Save current model state
print("Saving model checkpoint...")
save_success = save_model_checkpoint()

if save_success:
    print("Model checkpoint saved successfully!")
    print("You can download it from Google Drive: /MyDrive/TTS_Models/")